# Jenius
Forecasting for Jenius, digital bank in Indonesia

Forecast 5 models the following KPIs, that they'll use for strategic planning
- Balance
- Approved customers
- Funded customers
- New approved customers: shows the number of the new approved customers for each day. Once a customer downloads the bank's application, they need to get through the KYC ("know your customer") process, which means that a representative of a bank should see the customer at least once
- Balance forecast

Additional data sources:
- non-working days (weekends, national holidays)
- paydays
- dates of campaigns

Deliverables:
- models for the 5 KPIs
- fancy visualizations of the predictions
- raw data in Excel format

In [1]:
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import statsmodels.api as sm
from fbprophet import Prophet
from matplotlib import pyplot
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn import linear_model
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

random.seed(1)

sns.set(rc={'figure.figsize': (18, 6)})

Using TensorFlow backend.


In [2]:
def reduce_memory_usage(df, verbose=True): 
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
    start_mem = df.memory_usage().sum() / 1024**2 
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns: 
        col_type = df[col].dtypes 
        if col_type in numerics: 
            c_min = df[col].min() 
            c_max = df[col].max() 
            if str(col_type)[:3] == 'int': 
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max: 
                    df[col] = df[col].astype(np.int8) 
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max: 
                    df[col] = df[col].astype(np.int16) 
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max: 
                    df[col] = df[col].astype(np.int32) 
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max: 
                    df[col] = df[col].astype(np.int64)   
                else: 
                    if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max: 
                        df[col] = df[col].astype(np.float16) 
                    elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max: 
                        df[col] = df[col].astype(np.float32) 
                    else: 
                        df[col] = df[col].astype(np.float64)     
        end_mem = df.memory_usage().sum() / 1024**2 
        if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem)) 
        return df

In [3]:
def create_date_features(df, source_column, preposition):
    df[preposition + '_year'] = df[source_column].dt.year
    df[preposition + '_month'] = df[source_column].dt.month
    df[preposition + '_day'] = df[source_column].dt.day
    df[preposition + '_hour'] = df[source_column].dt.hour
    df[preposition + '_weekofyear'] = df[source_column].dt.weekofyear
    df[preposition + '_dayofweek'] = df[source_column].dt.dayofweek
    df[preposition + '_weekend'] = (df[source_column].dt.weekday >=5).astype(int)
    df[preposition + '_quarter'] = df[source_column].dt.quarter
    return df

## Exploratory Data Analysis

In [ ]:
df_input = pd.read_csv('input/input.csv', sep=';', parse_dates=['buss_date'], usecols=[1], engine='python')
print("{:,} records and {} features in train set.".format(df_input.shape[0], df_input.shape[1]))

df_input = reduce_memory_usage(df_input)

In [ ]:
df_input = create_date_features(df_input, 'buss_date', 'buss')

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
df_input[:3]

### Public holidays

In [ ]:
df_ph = pd.read_csv('input/indonesian_public_holidays_2018-19.csv', sep=';', parse_dates=['date'])
print("{:,} records and {} features in the public holiday set.".format(df_ph.shape[0], df_ph.shape[1]))

df_ph = reduce_memory_usage(df_ph)

In [ ]:
df_input_ph = df_input.merge(df_ph, left_on='buss_date', right_on='date', how='left')

In [ ]:
df_input.shape, df_input_ph.shape

In [ ]:
df_input_ph['is_holiday'] = df_input_ph['english_name'].isna() == False
df_input_ph['is_holiday'] = df_input_ph['is_holiday'].map({True: 1, False: 0})
df_input_ph.drop(['date', 'english_name'], axis=1, inplace=True)

Combining weekends with national holidays

In [ ]:
df_input_ph['is_workday'] = np.logical_or(df_input_ph['buss_dayofweek'] > 4, df_input_ph['is_holiday'])

In [ ]:
df_input_ph['is_workday'] = df_input_ph['is_workday'].map({True: 1, False: 0})

### Paydays

In [ ]:
df_pd = pd.read_csv('input/paydays.csv', sep=';', parse_dates=['date'])
print("{:,} records and {} features in the payday set.".format(df_ph.shape[0], df_ph.shape[1]))

df_pd = reduce_memory_usage(df_pd)

In [ ]:
df_input_ph = df_input_ph.merge(df_pd, left_on='buss_date', right_on='date', how='left')

In [ ]:
df_input_ph['is_payday'].fillna(0, inplace=True)
df_input_ph.drop(['date'], axis=1, inplace=True)

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
df_input_ph[['buss_date', 'balance', 'buss_dayofweek', 'is_workday', 'is_payday']][:5]

## Time series prediction

### Facebook's prophet
[Facebook's prophet](https://facebook.github.io/prophet/docs/quick_start.html#python-api)

In [ ]:
df_fbp = df_input_ph[['buss_date', 'balance']].rename(index=str, columns={"buss_date": "ds", "balance": "y"})

In [ ]:
df_fbp[:3]

In [ ]:
m = Prophet()
m.fit(df_fbp)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
forecast.to_csv('output/fbprophet_forecast.csv')

### Univariate LSTM
Inspired by [Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)

In [ ]:
df_input_ph['balance'] = df_input_ph['balance']

In [ ]:
df_input_ph = df_input_ph.fillna(method='bfill')
df_input_ph[['buss_date', 'balance']].to_csv('input/balance.csv', index=False)

In [ ]:
plt.figure(1, figsize=(15, 6))
plt.plot(df_input_ph['balance'])

In [ ]:
df_input_ph['balance_delta'] = df_input_ph['balance'].diff(1)

In [ ]:
plt.figure(1, figsize=(15, 6))
plt.plot(df_input_ph['balance_delta'])

#### Normalization

In [ ]:
df_input_ph = df_input_ph.fillna(method='bfill')

In [ ]:
df_input_ph[['buss_date', 'balance', 'balance_delta']][:5]

In [ ]:
df_input_ph[['buss_date', 'balance', 'balance_delta']].isna().sum()

Predicting the balance_delta

In [ ]:
predict_feature = 'balance' # balance_delta

scaler = MinMaxScaler(feature_range=(0, 1))
df_input_scaled = scaler.fit_transform(df_input_ph[[predict_feature]])

In [ ]:
df_input_scaled.shape

In [ ]:
train_size = int(len(df_input_scaled) * 0.67)
test_size = len(df_input_scaled) - train_size
train, test = np.array(df_input_scaled[0:train_size]), np.array(df_input_scaled[train_size:len(df_input_scaled)])

In [ ]:
len(train), len(test)

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=2)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
trainPredictPlot = np.empty_like(df_input_ph[[predict_feature]])
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(df_input_ph[[predict_feature]])
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(df_input_ph[[predict_feature]])-1, :] = testPredict

# plot baseline and predictions
plt.figure(1, figsize=(15, 6))
plt.plot(scaler.inverse_transform(df_input_ph[[predict_feature]]))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

### Multivariate LSTM
[Multivariate Time Series Forecasting with LSTMs in Keras](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/)

In [ ]:
groups = [11, 12, 16]
i = 1

pyplot.figure()
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(df_input_ph.values[:, group])
    pyplot.title(df_input_ph.columns[group], y=0.5, loc='right')
    i += 1
pyplot.show()

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
reframed = series_to_supervised(df_input_ph[['elapsed_days', 'is_workday', 'moving_avg_norm']], 1, 1)

In [ ]:
reframed[:3]

In [ ]:
train_size = int(len(df_input_ph) * 0.67)
test_size = len(df_input_ph) - train_size

train = reframed[:train_size].values
test = reframed[test_size:].values

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
#inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat * (ma_max - ma_min) + ma_min
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
#inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y * (ma_max - ma_min) + ma_min
inv_y = inv_y[:,0]

# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.plot(df_input_ph[['moving_avg']])
#plt.plot(trainPredictPlot)
plt.plot(inv_yhat)
plt.show()

## Regression
### Linear regression

In [ ]:
model = sm.OLS(df_input_ph['moving_avg'][14:232], df_input_ph['elapsed_days'][14:232], missing='drop').fit()
model.summary()

In [ ]:
df_input_ph['prediction'] = model.predict(df_input_ph['elapsed_days'])

In [ ]:
df_input_ph[['buss_date', 'elapsed_days', 'moving_avg', 'prediction']][0:10]

In [ ]:
mse = ((df_input_ph['moving_avg']-df_input_ph['prediction'])**2).sum()/100000000000000
print("MSE: {:,.2f}".format(mse))

#### Calculations from the Excel file

In [ ]:
df_input_ph['balance_delta'] = df_input_ph['balance'].diff(1)
df_input_ph['moving_avg'] = df_input_ph['balance_delta'].rolling(window=29, min_periods=24, center=True).mean()

In [ ]:
df_input_ph[['buss_date', 'elapsed_days', 'balance', 'balance_delta', 'moving_avg']][0:5]

In [ ]:
plt.subplot(111)
plt.plot(df_input_ph['elapsed_days'][14:232], df_input_ph['moving_avg'][14:232])

plt.show()